In [1]:
# Import Block

import pandas as pd
import datetime
import os
import tqdm

In [2]:
# Importing the Merged Train Data Sheet

train_CSV_path = '/Users/NorthCarpenter/Documents/Sort/Employment/Making Employment/Train Code/AMTRAC_2010-22.csv'

train_data = pd.read_csv(train_CSV_path)

In [3]:
#Looking at the Data + Deciding what to Do

train_data.head()

# Collumns to split
# - SCH DP
# - ACT DP
# - Coments
# - Origin Date

# Collumns to make
# - Date variable Relative to ____ /
# - Departure Delay Variable /
# - Adding a yesterdays average dellay variable 
# - Weekday variabe /

,Unnamed: 0,Train #,Origin Date,Sch Dp,Act Dp,Comments,Service Disruption,Cancellations,Station Code
0,0,140,12/31/2022 (Sa),12/31/2022 6:40 PM (Sa),6:45PM,Dp: 5 min late.,NaN,NaN,WFD
1,1,467,12/31/2022 (Sa),12/31/2022 6:32 PM (Sa),6:33PM,Dp: 1 min late.,NaN,NaN,WFD
2,2,488,12/31/2022 (Sa),12/31/2022 5:14 PM (Sa),5:26PM,Dp: 12 min late.,NaN,NaN,WFD
3,3,464,12/31/2022 (Sa),12/31/2022 3:14 PM (Sa),3:35PM,Dp: 21 min late.,NaN,NaN,WFD
4,4,463,12/31/2022 (Sa),12/31/2022 1:32 PM (Sa),1:33PM,Dp: 1 min late.,NaN,NaN,WFD


In [4]:
# Day of the week

week_days = train_data['Sch Dp'].str.slice(start = -3, stop = -1)
week_days

0           Sa
1           Sa
2           Sa
3           Sa
4           Sa
            ..
28947982    Tu
28947983    Tu
28947984    Tu
28947985    Tu
28947986    Tu
Name: Sch Dp, Length: 28947987, dtype: object

In [5]:
# Date variable

# Isolating date time
d_t = train_data['Sch Dp'].str.slice(start = 0, stop = -5)

# Making every time string include a date
def convert_dpt_datetime(date_str):
    return datetime.datetime.strptime(date_str, "%m/%d/%Y %I:%M %p")

d_t = d_t.apply(convert_dpt_datetime)

In [6]:
# Departure Delay Variable

# Removing any Arival data
def remove_Ar(comment_str):
    if str(comment_str).find('| ') != -1:
        return str(comment_str).split('| ')[1]
    else:
        return str(comment_str)

removed_Ar = train_data['Comments'].apply(remove_Ar)

# Splitting the remaining string by spaces for manipulation
broken_comment = removed_Ar.str.split(' ')

# Converting hours to min in all entreis
def hour_to_min(comment_lst):
    if 'hr' in comment_lst:
        hr = int(comment_lst[1])
        return hr * 60
    elif 'On' in comment_lst:
        return 0
    else:
        return 0
    
hours_in_min = broken_comment.apply(hour_to_min)

# Identifying just the minutes in all variations of data entry
def min_int(comment_lst):
    if 'and' in comment_lst:
        return int(comment_lst[4])
    elif 'hr' in comment_lst and 'min' in comment_lst:
        return int(comment_lst[3])
    elif 'On' in comment_lst:
        return 0
    elif 'min' in comment_lst:
        return int(comment_lst[1])
    else:
        return 0
    
mins = broken_comment.apply(min_int)

# Defining late as +, early as -, and on time as 0
def late_early(comment_lst):
    if 'late.' in comment_lst:
        return 1
    elif 'On' in comment_lst:
        return 0
    else:
        return -1

late_or_not = broken_comment.apply(late_early)

# Combinding them all into one list of delays in min
delay = (hours_in_min + mins) * late_or_not

In [7]:
# Adding Dummy Vars
train_data['Weekday'] = week_days
train_data['Date Time'] = d_t
train_data['Delay'] = delay

In [9]:
# Checking it all Worked
train_data.head(10)

,Unnamed: 0,Train #,Origin Date,Sch Dp,Act Dp,Comments,Service Disruption,Cancellations,Station Code,Weekday,Date Time,Delay,day_shifted_average_delay
0,0,140,12/31/2022 (Sa),12/31/2022 6:40 PM (Sa),6:45PM,Dp: 5 min late.,NaN,NaN,WFD,Sa,2022-12-31 18:40:00,5,7.908732
1,1,467,12/31/2022 (Sa),12/31/2022 6:32 PM (Sa),6:33PM,Dp: 1 min late.,NaN,NaN,WFD,Sa,2022-12-31 18:32:00,1,7.908732
2,2,488,12/31/2022 (Sa),12/31/2022 5:14 PM (Sa),5:26PM,Dp: 12 min late.,NaN,NaN,WFD,Sa,2022-12-31 17:14:00,12,7.908732
3,3,464,12/31/2022 (Sa),12/31/2022 3:14 PM (Sa),3:35PM,Dp: 21 min late.,NaN,NaN,WFD,Sa,2022-12-31 15:14:00,21,7.908732
4,4,463,12/31/2022 (Sa),12/31/2022 1:32 PM (Sa),1:33PM,Dp: 1 min late.,NaN,NaN,WFD,Sa,2022-12-31 13:32:00,1,7.908732
5,5,461,12/31/2022 (Sa),12/31/2022 11:36 AM (Sa),11:37AM,Dp: 1 min late.,NaN,NaN,WFD,Sa,2022-12-31 11:36:00,1,7.908732
6,6,460,12/31/2022 (Sa),12/31/2022 11:14 AM (Sa),11:15AM,Dp: 1 min late.,NaN,NaN,WFD,Sa,2022-12-31 11:14:00,1,7.908732
7,7,147,12/31/2022 (Sa),12/31/2022 9:41 AM (Sa),9:46AM,Dp: 5 min late.,NaN,NaN,WFD,Sa,2022-12-31 09:41:00,5,7.908732
8,8,450,12/31/2022 (Sa),12/31/2022 9:09 AM (Sa),9:14AM,Dp: 5 min late.,NaN,NaN,WFD,Sa,2022-12-31 09:09:00,5,7.908732
9,9,405,12/31/2022 (Sa),12/31/2022 8:27 AM (Sa),8:30AM,Dp: 3 min late.,NaN,NaN,WFD,Sa,2022-12-31 08:27:00,3,7.908732


In [10]:
# Summary Stats
train_data.describe()

,Unnamed: 0,Train #,Date Time,Delay,day_shifted_average_delay
count,2.894612e+07,2.894612e+07,28946115,2.894612e+07,2.894612e+07
mean,7.279747e+03,4.378551e+02,2016-06-20 01:17:15.262543104,1.096482e+01,1.094718e+01
min,0.000000e+00,1.000000e+00,2010-01-02 00:07:00,-3.600000e+02,2.665186e+00
25%,1.227000e+03,8.700000e+01,2013-08-14 12:09:00,0.000000e+00,7.468382e+00
50%,3.770000e+03,3.060000e+02,2016-06-15 12:34:00,3.000000e+00,8.803653e+00
75%,9.224000e+03,6.460000e+02,2019-03-20 07:28:00,1.100000e+01,1.261824e+01
max,5.807900e+04,9.998000e+03,2022-12-31 23:59:00,2.160000e+03,4.986985e+01
std,9.391755e+03,5.056963e+02,NaN,2.872914e+01,5.632947e+00


In [11]:
# Save Modified PDF!!!!
train_project_folder = os.path.abspath('/Users/NorthCarpenter/Documents/Sort/Employment/Making Employment/Train Code/cleanAllTrainData.csv')

train_data.to_csv(train_project_folder)